![header](../figures/logos_partenaires._cmems_se.jpg)

# Benchmark SWOT Swath Filtering

<div style="text-align: right"><i> 2022-08-24 CNN_BENCHMARK_DEMO </i></div>

***
**Authors:**  CLS & Datlas <br>
**Copyright:** 2022 CLS & Datlas <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h1><center>Benchmark of CNN filter</center></h1></div>

***
**General Note 1**: Execute each cell through the <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button from the top MENU (or keyboard shortcut `Shift` + `Enter`).<br>
<br>
**General Note 2**: If, for any reason, the kernel is not working anymore, in the top MENU, click on the <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Then, in the top MENU, click on "Cell" and select "Run All Above Selected Cell".<br>
***


<div class="alert alert-danger" role="alert">

<h3>Learning outcomes</h3>

At the end of this notebook you will know:
<ul>
  <li>How you can display unfiltered, filtered SWOT Level3 SSH products.</li>
  <li>How you can evaluate the cnn filter: statistical and spectral analysis.</li>
</ul>
    
</div>



In [ ]:
import os
import glob
import sys
sys.path.append('..')
from src.benchmark import *
from src.plot import *
from src.swot import *

<div class="alert alert-info" role="alert">

<h2>1. Input files to analyse</h2>

</div>

## Data challenge inputs

In [ ]:
# Check if inputs have already been downloaded otherwise do
if not os.path.isdir('../dc_inputs'):
    # Get pseuso-observation
    !wget https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/ocean-data-challenges/2022a_SWOT_karin_error_filtering/dc_inputs.tar.gz
    !tar -xvf dc_inputs.tar.gz --directory ../
    !rm -f dc_inputs.tar.gz

In [ ]:
list_of_file_inputs = sorted(glob.glob('../dc_inputs/input_ssh_karin_013*.nc'))

## Filtered outputs to evaluate

In [ ]:
list_of_file = sorted(glob.glob('../inputs/eNATL60-BLB002/cnn/SWOT_L2_LR_Filtered_013*.nc'))

<div class="alert alert-info" role="alert">

<h2>2. Illustration on one pass (same as quickstart example)</h2>

</div>

In [ ]:
plot_demo_pass(list_of_file_inputs[3], list_of_file[3])

<div class="alert alert-info" role="alert">

<h2>3. Statistical & Spectral Analysis </h2>

</div>

## 3.1 Parameters

In [ ]:
etuvar = 'ssh_karin_filt'
filter_name = 'CNN'
output_dir = '../results'
stat_output_filename = f'{output_dir}/stat_CNN.nc'
stat_by_regime_output_filename = f'{output_dir}/stat_by_regime_CNN.nc'
psd_output_filename = f'{output_dir}/psd_CNN.nc'
cnn_bench = Benchmark()

## 3.2 Compute grid boxes statistics

In [ ]:
%%time
# Compute gridded stats
cnn_bench.compute_stats(list_of_file, etuvar, list_of_file_inputs)

# Write gridded stats 
cnn_bench.write_stats(stat_output_filename, filter=filter_name)

# Plot gridded stats
cnn_bench.display_stats(stat_output_filename, xlim=(-100, 50), ylim=(0, 80))

## 3.3 Compute statistics by regime (coastal, offshore low variability, offshore high variability)

In [ ]:
# To speed up computation
# from dask.distributed import Client
# client = Client(n_workers=8, threads_per_worker=8, memory_limit='8GB')

In [ ]:
%%time 
# Compute stats by regime
cnn_bench.compute_stats_by_regime(list_of_file, etuvar, list_of_file_inputs)

# Write stats by regime
cnn_bench.write_stats_by_regime(stat_by_regime_output_filename, filter=filter_name) 

# Plot stats by regime
cnn_bench.plot_stats_by_regime(stat_by_regime_output_filename)

In [ ]:
#client.close()

## 3.4 Compute Spectral scores

In [ ]:
%%time
# Compute PSD scores
cnn_bench.compute_along_track_psd(list_of_file, etuvar,list_of_file_inputs, lengh_scale=1000)

# Write PSD scores
cnn_bench.write_along_track_psd(psd_output_filename, filter=filter_name)

# Plot PSD scores
cnn_bench.display_psd(psd_output_filename)

<div class="alert alert-info" role="alert">

<h2>4. Leaderboard </h2>

</div>

In [ ]:
cnn_bench.summary('demo_benchmark_CNN.ipynb')